#### from functools import cache

## cache / lru_cache:
- 记录以往函数运行的结果，避免参数重复时反复调用，达到提高性能的作用，相同参数会调用缓存

```
maxsize : 最大缓存个数，超过时将最近最少被访问的元素移除，从而腾出空间给新的元素

type :true:不同类型的参数分别缓存，如3.0和3  False:3.0和3都是同一个

```

In [7]:
from functools import lru_cache


@lru_cache(maxsize=None)
def test_(a,b):
    print("已经被调用")

print(test_(1,2))
print(test_(1,2)) # 第二次调用缓存


test_list=[]
@lru_cache(maxsize=None)
def test_value(val):
    test_list.append(val)
    print(f'test_list append :{val}')
    for index,value in enumerate(test_list):
        print(index,value)

test_value(2)
test_value(2) # 第二次直接调用缓冲运行的结果，而不会再次执行append
print(test_list)

已经被调用
None
None
test_list append :2
0 2
[2]


#### cached_property:
- 设置为类属性 equal to @property
- 设置该属性为缓存,存在于整个实例周期中


In [2]:
# from functools import cached_property
import functools
class DataSet:
    def __init__(self, sequence_of_numbers):
        self._data = sequence_of_numbers


    @functools.cached_property
    def test_called(self):
        print(f'>> method is called')
        return 1

    # @cached_property
    # def variance(self):
    #     return statistics.variance(self._data)

dataset=DataSet([1,1,1,1])
# 第一次调用
dataset.test_called()
# 第二次不会去调用，而是调用缓存
dataset.test_called()

AttributeError: module 'functools' has no attribute 'cached_property'

#### cmp_to_key(func)
可以将一个cmp函数变成一个key函数，从而支持自定义排序,主要和sort（）搭配使用

In [9]:
lst = [(9, 4), (2, 10), (4, 3), (3, 6)]
lst.sort(key=lambda item: item[0])
print(lst)

#  等价于

from functools import cmp_to_key
lst = [(9, 4), (2, 10), (4, 3), (3, 6)]

def cmp(x, y):
    """如果元组里第一个元素是奇数，就用元组里第一个元素进行排序，
    如果元组里第一个元素是偶数，则用这个元组里的第二个元组进行大小比较"""
    a = x[0] if x[0] %2 == 1 else x[1]
    b = y[0] if y[0] %2 == 1 else y[1]

    return 1 if a > b else -1 if a < b else 0

lst.sort(key=cmp_to_key(cmp))
print(lst)


[(2, 10), (3, 6), (4, 3), (9, 4)]
[(4, 3), (3, 6), (9, 4), (2, 10)]


#### total_ordering(cls)
- 类装饰器，给类添加比较的方法
- 被修饰的类必须至少定义 __lt__(), __le__(), __gt__() 或 __ge__() 中的一个，同时，被修饰的类还应该提供 __eq__() 方法。
  类中定义了两个比较方法：一个定义相等，一个定义顺序。其他比较方法由@total_ordering基于这两个定义完成，
  包括__le__()、__gt__()和__ge__()等，不等比较方法__ne__()默认基于__eq__()生成，装饰器无须参与
#### 类的比较
- __eq__()：定义的两个类相同的比较方法
- __lt__(),__gt__() :定义类小于,大于的比较方法
- __le__(),__ge__():<= , >=
- __ne__(): "!="



In [ ]:
from functools import total_ordering
from typing import NamedTuple ,Any
# namedtuple是继承自tuple的子类。
# namedtuple创建一个和tuple类似的对象，而且对象拥有可访问的属性。

@total_ordering
class Card2(NamedTuple):
    rank: int
    suit: str
    def __eq__(self, other: Any) -> bool:
        if isinstance(other, Card2):
            return self.rank == other.rank
        elif isinstance(other, int):
            return self.rank == other
        return NotImplemented
    def __lt__(self, other: Any) -> bool:
        if isinstance(other, Card2):
            return self.rank < other.rank
        elif isinstance(other, int):
            return self.rank < other
        return NotImplemented




#### partial
-  它返回一个偏函数对象，这个对象和 func 一样，可以被调用，同时在调用的时候可以指定位置参数 (args) 和 关键字参数(*kwargs)。
如果有更多的位置参数提供调用，它们会被附加到 args 中。如果有额外的关键字参数提供，它们将会扩展并覆盖原有的关键字参数。
简单说就是给某些参数一些固定值


#### particalmethod
- 对于类方法，因为第一个参数是self，使用partial就会报错了。 对此，python3.4新引入了 partialmethod，


- 必须指明参数名
(param=value) √    
(value) ×

In [20]:
from functools import partial

def add(x1,x2,x3):
    return x1+x2+x3

print(add(1,2,3))
add_X1_2 = partial(add,x1=3)
print(add_X1_2(x2=1,x3=2))
print(add_X1_2(x2=2,x3=3))

def add(*args, **kwargs):
    # 打印位置参数
    print("-"*20)
    for n in args:
        print(n)
    print("-"*20)
    # 打印关键字参数
    for k, v in kwargs.items():
       print('%s:%s' % (k, v))

print(add(1, 2, 3, v1=10, v2=20))

## partial
add_partial = partial(add, 10, k1=10, k2=20) # 固定了10,k1,k2两个参数
add_partial(1, 2, 3, k3=20)
add_partial(1, 2, 3, k1=20)


### partical
class mathmethod():

    def add(self,x,y):
        return x+y

    def add_x_1(self,y):
        add_= partial(self.add,x=1)
        return add_(y=y)
test = mathmethod()
print(test.add_x_1(y=2))

6
6
8
--------------------
1
2
3
--------------------
v1:10
v2:20
None
--------------------
10
1
2
3
--------------------
k1:10
k2:20
k3:20
--------------------
10
1
2
3
--------------------
k1:20
k2:20
3
